### 使用routechain来实现判断存款还是取款

In [10]:
from langchain.chains.router import MultiPromptChain
from langchain_ollama import OllamaLLM
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [11]:
llm = OllamaLLM(model="llama3.1:latest",temperature=0.9)


In [ ]:
destination_chains = {}
# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [13]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

In [14]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE


In [15]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [17]:
print(destinations)
print(destinations_str)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']
物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [18]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [19]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [26]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [27]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑体辐射的定义和原理?'}
> Finished chain.
{'input': '黑体辐射的定义和原理?', 'text': '好问题！黑体辐射是一种非常重要的物理现象，涉及到热辐射和热平衡。让我们简单来说：\n\n**什么是黑体辐射？**\n\n黑体辐射是指物体在绝热条件下（即没有外界能量交换）辐射出的全部电磁辐射的总和。这意味着，在完全吸收所有进入其辐射场的辐射后，物体所辐射出的一切都是黑体辐射。\n\n**原理是这样的：**\n\n当物体温度升高时，它会发出更多的能量。由于热运动引起电子和原子跃迁，并释放能量，成为光学辐射。这包括从红外辐射到紫外辐射的所有波长。\n\n黑体辐射是由斯汀贝格（Stefan）-波尔茨曼定律描述的，这个定律将物体表面吸收和发射辐射的能力与其温度相关联。这个定律可以推导出，物体辐射出的能量总量与其温度的四次方成正比。\n\n简而言之，黑体辐射是描述热辐射最基本的概念之一，是理解热力学和电磁学之间关系的关键。这是一个非常重要的物理现象，不仅在理论上有意义，也在实际应用中发挥着重要作用，如光谱分析、探测天文学等。\n\n你想知道更多吗？'}


In [22]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '找出大于40且其加一可被3整除的第一个质数'}
> Finished chain.
{'input': '找出大于40且其加一可被3整除的第一个质数', 'text': '让我们一步步地分解这个问题：\n\n1. 首先，我们需要找到大于 40 的数字。\n2. 其次，我们需要找到这些数字中，其加一可以被 3 整除的数字。也就是说，数字加一后，余数为 0 时，可以被 3 整除。\n3. 最后，我们需要在这些符合条件的数字中找出第一个质数。\n\n首先，让我们找出大于 40 的数字。这些数字包括：41、42、43、...。\n\n然后，让我们找到这些数字中的加一可被 3 整除的数字。这意味着，每个数字加 1 后，余数为 0 时，可以被 3 整除。\n\n例如：\n- 41 加 1 等于 42，不能被 3 整除。\n- 42 加 1 等于 43，不能被 3 整除。\n- 43 加 1 等于 44，不能被 3 整除。\n- ...\n\n继续这个过程，我们发现，当数字为 40 时，加一等于 41（不能被 3 整除），然后是 41、42、43 和 44。我们注意到，这些数字加一都不是 3 的倍数。\n\n当数字为 47 时，加一等于 48（可以被 3 整除），但 47 不是质数，因为它有两个不同的正因数：1 和 47。\n\n继续这个过程，我们发现，当数字为 53、55、58、61、62、64 等时，加一也都不是 3 的倍数。其中，53 是第一个符合条件的质数，因为它是仅有的两个不同正因数：1 和 53。\n\n因此，我们找到的第一个大于 40 且其加一可被 3 整除的质数是 **53**。'}


In [23]:
router_chain.verbose = True

In [24]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是一种什么样的天体?'}
> Finished chain.
{'input': '黑洞是一种什么样的天体?', 'text': '黑洞就是一种极重的天体，质量非常大，以致其引力足以将包括光在内的所有物质和能量全部吸收。'}
